In [78]:
import urllib  
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

#importing required packages

In [79]:
opener=urllib.request.build_opener() 
opener.addheaders=[('User-Agent', 'Google Chrome/57.0.2987.133')]

In [80]:
url=('http://www.trademe.co.nz')
ourURL=opener.open(url).read()
import re

In [81]:
soup=BeautifulSoup(ourURL, "lxml")

In [82]:
#soup #this is all the HTML on the given URL.

#print(soup(text=re.compile("Mobile")))

In [83]:
def urlmaker(page):
    return('http://www.trademe.co.nz/browse/categorylistings.aspx?rptpath=344-422-&mcatpath=mobile-phones%2fmobile-phones&page='
           +str(page))
urlmaker(0)

'http://www.trademe.co.nz/browse/categorylistings.aspx?rptpath=344-422-&mcatpath=mobile-phones%2fmobile-phones&page=0'

In [84]:
def soupify(page):
    return(BeautifulSoup(opener.open(urlmaker(page)).read(),"lxml") )
#soupify(0)

In [85]:
#this is every instance of soupify, to make less reliance on functions later-> and to hault time.
#I will run this once per day.

soups=[]

for i in range(0,99):
    soups.append(soupify(i))

#soups[0]

#UPDATE: Yeah this made it work like 10X faster. :>

In [86]:
#definitions have to be more complex here...

#the below contains all items as a single string.

def item(page):
    p=soups[page].findAll('a', {'class':re.compile("tile")})
    paragraphs=[]
    
    for x in p:
        paragraphs.append(str(x))
    for i in range(len(paragraphs)):  
        paragraphs[i]=" ".join(paragraphs[i].split())
    return(paragraphs)

#item(0)

#This is information on each item for that page. If I can find an effecive way to
#subset this, I can gather information in 1 step

In [87]:
#TIME TO GO HYPERSPEED.

items=[]
for j in range(len(soups)):
        items.append(item(j))
     

In [88]:
def location(page):
    liste=[]
    for i in range(len(items[page])):
        liste.append(re.search('<div class="location">(.+?)</div>', items[page][i]).group(1)) 
    return(liste) 

def name(page):
    liste=[]
    for i in range(len(items[page])):
        liste.append(re.search('<div class="title">(.+?)</div>', items[page][i]).group(1))  
    return(liste)

def bidprices(page):
    liste=[]
    for i in range(len(items[page])):
        try:
            liste.append(re.search('<div class="listingBidPrice" id="SuperGridGallery_ctl00_BidInfo_listingBidPrice">(.+?)</div>', items[page][i]).group(1))
        except:
            liste.append('BuyNowOnly')
    return(liste)

def buynowprices(page):
    liste=[]
    for i in range(len(items[page])):
        try:
            liste.append(re.search('<div class="listingBuyNowPrice" id="SuperGridGallery_ctl00_BuyNow_listingBuyNowPrice">(.+?)</div>', items[page][i]).group(1))  
        except:
            liste.append('BidOnly')
    return(liste)

'Enter logic either side of (.+?) {in the red}'

'Enter logic either side of (.+?) {in the red}'

In [89]:
locations=[]

for i in range(len(soups)):
    locations.append(location(i))
    
flattened_locations = [val for sublist in locations for val in sublist]
#flattened_locations

In [90]:
names=[]

for i in range(len(soups)):
    names.append(name(i))
    
flattened_names = [val for sublist in names for val in sublist]
#flattened_names

In [91]:
bidpricess=[]

for i in range(len(soups)):
    bidpricess.append(bidprices(i))
    
flattened_bidprices = [val for sublist in bidpricess for val in sublist]
#flattened_bidprices

In [92]:
buynowpricess=[]

for i in range(len(soups)):
    buynowpricess.append(buynowprices(i))
    
flattened_buynowprices = [val for sublist in buynowpricess for val in sublist]
#flattened_buynowpricess

In [93]:
Frame=pd.DataFrame()

Frame['Location']=flattened_locations
Frame['Item_Name']=flattened_names
Frame['Bid_Price']=flattened_bidprices
Frame['BuyNow_Price']=flattened_buynowprices

Frame.to_csv('TM_scrape_initial.csv', delimiter=',', encoding='utf-8')